# Imports

In [ ]:
%env CUDA_VISIBLE_DEVICES=1

In [ ]:
import os
import torch
from torch import nn
import torchxrayvision as xrv
import matplotlib.pyplot as plt

In [ ]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'

In [ ]:
import pandas as pd
pd.options.display.max_columns = None

# Load model

## Compiled model

In [ ]:
%run ../models/checkpoint/__init__.py

In [ ]:
run_name = '1215_174443_cxr14_resnet-50-v2_lr0.0001_os_Cardiomegaly_normS_size256_sch-roc_auc-p5-f0.1'
debug_run = False

In [ ]:
compiled_model = load_compiled_model_classification(run_name, debug=debug_run)
compiled_model.metadata['model_kwargs']

## XRV model

In [ ]:
xrv_model = xrv.models.DenseNet(weights="nih", in_channels=1)
xrv_model

In [ ]:
print(super(type(xrv_model), xrv_model).__repr__())

In [ ]:
type(xrv_model)

In [ ]:
xrv.models.DenseNet

In [ ]:
class XRVWrapper(nn.Module):
    """Wrapper for TorchXrayVision models, to comply with own API."""
    def __init__(self, model):
        super().__init__()
        self.model = model
        
        assert isinstance(model, xrv.models.DenseNet), (
            f'wrapped model should be instance of xrv.models.DenseNet, got {type(model)}',
        )
        
        self.valid_idxs, self.valid_pathologies = zip(*[
            (index, pat)
            for index, pat in enumerate(model.pathologies)
            if pat
        ])
        
        self.valid_idxs = torch.tensor(self.valid_idxs)
        
    def forward(self, x):
        output = self.model(x)
        # shape: bs, n_total_pathologies
        
        output = output.index_select(dim=-1, index=self.valid_idxs.to(output.device))
        # shape: bs, n_valid_pathologies
        
        return (output,)

In [ ]:
model = XRVWrapper(xrv_model)
model

In [ ]:
model = model.to('cuda')
model

# Load data

In [ ]:
%run ../datasets/__init__.py

In [ ]:
dataset_kwargs = {
    'dataset_name': 'cxr14',
    'dataset_type': 'test-bbox',
    'max_samples': None,
    'image_format': 'L', # XRV accepts 1 channel
}
dataloader = prepare_data_classification(**dataset_kwargs)
dataset = dataloader.dataset
len(dataset)

In [ ]:
from PIL import Image
import numpy as np
from torchvision import transforms

In [ ]:
image_fpath = os.path.join(dataset.image_dir, dataset.label_index['FileName'][1])

In [ ]:
image = Image.open(image_fpath).convert('RGB')
image.size

In [ ]:
image_np = np.array(image)
image_np.shape

In [ ]:
image_np.min(), image_np.max()

In [ ]:
# image_tensor = dataset.transform(image)
image_tensor = transforms.ToTensor()(image)
image_tensor.size()

In [ ]:
image_tensor.min(), image_tensor.max()

In [ ]:
dataset.transform

In [ ]:
plt.imshow(image)

# Evaluate in dataset

In [ ]:
%run -n ../eval_classification.py

In [ ]:
metrics = evaluate_model(model, dataloader)
metrics